In [91]:
import os
from langchain.embeddings import OpenAIEmbeddings

import pprint
from tree_sitter import Language, Parser
import pandas as pd
import openai
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
from IPython.display import display

EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"
pp = pprint.PrettyPrinter(indent=4)

In [211]:
def ask(df, query, updatestep, n=3):
    updatestep("UNDERSTANDING QUERY")
    response = openai.ChatCompletion.create(
        temperature=0,
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You provide concise code search queries"},
            {"role": "user", "content": "Code search for the following: What framework does this appear to use?"},
            {"role": "assistant", "content": "framework and tools"},
            {"role": "user", "content": "Code search for the following: How does the app start up?"},
            {"role": "assistant", "content": "Start initialization"},
            {"role": "user", "content": "Code search for the following: What javascript framework is being used?"},
            {"role": "assistant", "content": "Javascript framework used"},
            {"role": "user", "content": "Code search for the following: How is user authentication handled?"},
            {"role": "assistant", "content": "User authentication"},
            {"role": "user", "content": "Code search for the following: How is routing handled in the application?"},
            {"role": "assistant", "content": "Routing"},
            {"role": "user", "content": "Code search for the following: How does the application handle errors and exceptions?"},
            {"role": "assistant", "content": "Errors and exception handling"},
            {"role": "user", "content": "Code search for the following: " + query},
        ],
    )
    query_search = response["choices"][0]["message"]["content"]
    updatestep("QUERY UNDERSTOOD: " + query_search)
    updatestep("SEARCHING CODEBASE")
    query_embedding = get_embedding(query_search, engine=EMBEDDING_MODEL)
    df['similarities'] = df.source_embedding.apply(
        lambda x: cosine_similarity(x, query_embedding))
    res = df.sort_values('similarities', ascending=False).head(n)

    introduction = 'Use the below code snippets to answer questions about the project. If the answer cannot be found, write "I could not find an answer.". \n \n'
    question = f"\n\nQuestion: {query}. Provide relevant code snippets."
    message = introduction

    for r in res.iterrows():
        message += r[1]['node_source'] + "\n"

    messages = [
        {"role": "system", "content": "You are a helpful assistant that answers questions about codebases"},
        {"role": "user", "content": message + question},
    ]

    updatestep("TALKING TO CHAT GPT")
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [210]:
answer = ask(df, "How is authentication being impelmented?", lambda x: print(x), n=11)
pp.pprint(answer)

UNDERSTANDING QUERY
QUERY UNDERSTOOD Authentication implementation
SEARCHING CODEBASE
TALKING TO CHAT GPT
('Authentication is being implemented using Auth0. The relevant code snippets '
 'are:\n'
 '\n'
 '1. In the `AuthenticatedRepositoryInput` component, the `isAuthenticated` '
 'state variable is being used to determine whether the user is authenticated '
 'or not. If the user is not authenticated, a prompt is displayed to authorize '
 'Adrenaline to access the list of repositories. If the user is authenticated, '
 'the list of repositories is displayed.\n'
 '\n'
 '2. In the `UserNavDropdown` component, the `useAuth0` hook is being used to '
 "get the user's access token and profile information. The `onLogout` function "
 'is passed as a prop to handle logging out the user.')


In [100]:
df = pd.read_pickle('./embeddings/adrenaline.pkl')

In [101]:
df

,node_type,node_source,relative_path,source_embedding
0,lexical_declaration,"nst UserNavDropdown = ({ onLogout, setShowSubs...",Header.js,"[-0.009951675881428873, -0.007107434536750538,..."
1,class_declaration,class AuthenticatedRepositoryInput extends Com...,AuthenticatedRepositoryInput.js,"[-0.008377281659314235, 0.01707976811192554, -..."
2,class_declaration,class PaywallMessage extends Component {\n ...,PaywallMessage.js,"[-0.0002918101964744845, 0.010486406278300301,..."
3,class_declaration,class RepositoryInput extends Component {\n ...,RepositoryInput.js,"[0.007334597124679949, 0.007334597124679949, 0..."
4,function_declaration,function MinusSquare(props) {\n return (\n ...,FileStructure.js,"[-0.010450839245885177, 0.015845662257130553, ..."
...,...,...,...,...
290,expression_statement,"mixpanel.init(MIXPANEL_TOKEN, { debug: true, i...",mixpanel.js,"[-0.016765276587535297, -9.59411371558793e-05,..."
291,lexical_declaration,let actions = {\n\tidentify: (id) => {\n\t\tmi...,mixpanel.js,"[-0.014205550327047447, 0.0012967238026679808,..."
292,lexical_declaration,const Mixpanel = actions;,mixpanel.js,"[-0.005957620342156769, 0.0027311414581901537,..."
293,function_declaration,"function MyApp({Component, pageProps}) {\n ...",_app.js,"[0.00888381300796254, 0.021216410216596614, -0..."
